## Importing the Required Libraries

https://deepai.org/publication/general-purpose-long-context-autoregressive-modeling-with-perceiver-ar

In [5]:
import gzip
import random

import numpy as np
import torch
import torch.optim as optim
import tqdm
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

from perceiver_ar_pytorch import PerceiverAR
from perceiver_ar_pytorch.autoregressive_wrapper import AutoregressiveWrapper

from sklearn.model_selection import train_test_split

import re

from pychord import Chord
import json

import pickle

## Defining Constants and Helper Functions

In [2]:
# constants

NUM_BATCHES = int(1e4)
BATCH_SIZE = 64
GRADIENT_ACCUMULATE_EVERY = 4
LEARNING_RATE = 1e-4
VALIDATE_EVERY = 100
GENERATE_EVERY = 500
GENERATE_LENGTH = 2
SEQ_LEN = 4
PREFIX_SEQ_LEN = 1

In [3]:
# helpers

def cycle(loader):
    while True:
        for data in loader:
            yield data


def decode_token(token):
    return str(chr(max(32, token)))


def decode_tokens(tokens):
    return "".join(list(map(decode_token, tokens)))

'''
model = PerceiverAR(
    num_tokens = 256,
    dim = 512,
    depth = 8,
    heads = 8,
    dim_head = 64,
    cross_attn_dropout = 0.5,
    max_seq_len = SEQ_LEN,
    cross_attn_seq_len = PREFIX_SEQ_LEN
)
'''

model = PerceiverAR(
    num_tokens = 121, #974 if not ignoring the different bass note
    dim = 128,
    depth = 16,
    heads = 8,
    dim_head = 64,
    cross_attn_dropout = 0.6,
    max_seq_len = SEQ_LEN,
    cross_attn_seq_len = PREFIX_SEQ_LEN
)


model = AutoregressiveWrapper(model)
model.cuda()

AutoregressiveWrapper(
  (net): PerceiverAR(
    (token_emb): Embedding(121, 128)
    (pos_emb): Embedding(4, 128)
    (rotary_pos_emb): RotaryEmbedding()
    (perceive_layers): ModuleList(
      (0): ModuleList(
        (0): CausalPrefixAttention(
          (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (context_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
          (to_q): Linear(in_features=128, out_features=512, bias=False)
          (to_kv): Linear(in_features=128, out_features=1024, bias=False)
          (to_out): Linear(in_features=512, out_features=128, bias=True)
        )
        (1): Sequential(
          (0): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (1): Linear(in_features=128, out_features=512, bias=False)
          (2): GELU(approximate=none)
          (3): Dropout(p=0.0, inplace=False)
          (4): Linear(in_features=512, out_features=128, bias=False)
    

## Loading and Preprocessing the Data

### An approach with own embeddings (that specify the notes that make up a chord)

In [4]:
temp = open('data/soul_chords.txt','r').read().splitlines()

In [5]:
# Load the lookup for pitch to number translation
with open('data/pitch_embedding.json') as d:
    pitches = json.load(d)

In [6]:
# Replace the diminished and augmented chord notation to work with the PyChord library
chords = [x.replace('o', 'dim') for x in temp]
chords = [x.replace('+', 'aug') for x in chords]

#### Ignoring the different bass notes (i.e., assuming the root is in the base) to reduce chord variety

In [10]:
chords = [re.sub('/.*', '', x) for x in chords]

In [11]:
def embed_chord(chord):
  if chord == 'N':
    return [0.0, 0.0, 0.0, 0.0, 0.0]
  x = chord
  if 'E#' in x:
    x = x.replace("E#", "F")
  if 'B#' in x:
    x = x.replace("B#", "C")
  try:
    x = [int(pitches.get(item,item))  for item in Chord(x).components_with_pitch(root_pitch=3)]
    x += [0] * (5 - len(x))
  except:
    x= [0,0,0,0,0]
  return x

In [13]:
len(set(chords)) #unique chords

120

In [14]:
embeddings_index = {ch: embed_chord(ch) for ch in list(set(chords))}
list(embeddings_index.items())[0] # an arbitrary example

('G', [8, 12, 15, 0, 0])

In [15]:
# a reverse lookup function to get chords from embeddings 
inv_embeddings = {" ".join([str(x) for x in v]): k for k, v in embeddings_index.items()}

In [49]:
inv_embeddings

{'12 16 19 22 0': 'B7',
 '5 9 13 0 0': 'Eaug',
 '9 13 17 0 0': 'G#aug',
 '3 8 10 0 0': 'Dsus4',
 '11 14 17 20 0': 'A#dim7',
 '3 7 10 14 0': 'DM7',
 '10 14 18 0 0': 'Aaug',
 '2 5 8 0 0': 'C#dim',
 '12 15 19 0 0': 'Bm',
 '4 9 11 0 0': 'D#sus4',
 '10 14 17 20 0': 'A7',
 '5 9 12 0 0': 'E',
 '8 12 15 0 0': 'G',
 '2 6 9 13 0': 'C#M7',
 '5 9 12 15 0': 'E7',
 '10 14 17 21 0': 'AM7',
 '3 7 11 0 0': 'Daug',
 '6 9 13 16 0': 'Fm7',
 '1 6 8 0 0': 'Csus4',
 '12 16 20 0 0': 'Baug',
 '9 12 15 0 0': 'G#dim',
 '1 3 8 0 0': 'Csus2',
 '4 7 10 0 0': 'D#dim',
 '7 10 13 0 0': 'F#dim',
 '2 4 9 0 0': 'C#sus2',
 '7 11 14 0 0': 'F#',
 '6 9 13 0 0': 'Fm',
 '7 10 13 16 0': 'F#dim7',
 '11 15 19 0 0': 'A#aug',
 '6 9 12 15 0': 'Fdim7',
 '1 5 9 0 0': 'Caug',
 '4 8 12 0 0': 'D#aug',
 '4 8 11 15 0': 'D#M7',
 '5 9 12 16 0': 'EM7',
 '11 13 18 0 0': 'A#sus2',
 '9 13 16 0 0': 'G#',
 '12 15 19 22 0': 'Bm7',
 '11 14 18 21 0': 'A#m7',
 '9 12 16 19 0': 'G#m7',
 '11 14 17 0 0': 'A#dim',
 '5 7 12 0 0': 'Esus2',
 '8 11 14 17 0': '

In [16]:
chords_emb = [embed_chord(ch) for ch in chords]

In [17]:
chords_emb = np.array(chords_emb, dtype=np.uint8)
chords_emb

array([[11, 15, 18, 21,  0],
       [ 6,  9, 13, 16,  0],
       [ 1,  4,  8, 11,  0],
       ...,
       [ 6, 10, 13, 17,  0],
       [11, 15, 18, 22,  0],
       [11, 15, 18,  0,  0]], dtype=uint8)

In [18]:
trX, vaX = train_test_split(chords_emb, test_size=0.3, random_state=42)
data_train, data_val = torch.from_numpy(trX), torch.from_numpy(vaX)

In [20]:
def decode_tokens(tokens):
    tmp = tokens.tolist()
    chords = []
    for ch in tmp:
        chords.append(" ".join([str(x) for x in ch]))
    chords = [inv_embeddings[x] for x in chords]
    
    return chords

### An approach without custom embeddings (letting the model do its thing)

In [6]:
temp = open('data/soul_chords.txt','r').read().splitlines()
temp = [re.sub('/.*', '', x) for x in temp]

lookup_set = list(set(temp))
lookup_nr = list(range(0,len(lookup_set)))
lookup = {lookup_set[nr]: nr for nr in lookup_nr}

inv_lookup = {v: k for k, v in lookup.items()}

X = [lookup[x] for x in temp]
trX, vaX = train_test_split(np.array(X, dtype=np.uint8), test_size=0.30, random_state=42)
data_train, data_val = torch.from_numpy(trX), torch.from_numpy(vaX)

In [8]:
# Load the dataset (chords from MIDIs) in pickle format
chords = pickle.load(open("data/soul_chords.pickle", "rb"))

chords = [[x[0].replace('o', 'dim') for x in song] for song in chords]
chords = [[x.replace('+', 'aug') for x in song] for song in chords]

random.shuffle(chords)

chords = [x for y in chords for x in y]

temp  = [re.sub('/.*', '', x) for x in chords]

lookup_set = list(set(temp))
lookup_nr = list(range(0,len(lookup_set)))
lookup = {lookup_set[nr]: nr for nr in lookup_nr}

inv_lookup = {v: k for k, v in lookup.items()}
X = [lookup[x] for x in temp]

split = int(len(X)*0.8)
trX, vaX = np.array(X[0:split], dtype=np.uint8),np.array(X[split:], dtype=np.uint8)
data_train, data_val = torch.from_numpy(trX), torch.from_numpy(vaX)

In [8]:
chords = [[x[0].replace('o', 'dim') for x in song] for song in chords]
chords = [[x.replace('+', 'aug') for x in song] for song in chords]

In [9]:
print(len(set(temp)))

120


In [10]:
print(data_train[0], len(data_train))
print(data_val[0], len(data_val))

tensor(41, dtype=torch.uint8) 134909
tensor(12, dtype=torch.uint8) 33728


In [11]:
def decode_tokens(tokens):
    tmp = tokens.tolist()
    chords = [inv_lookup[x] for x in tmp]
    
    return chords

## Training Setup

In [12]:
class TextSamplerDataset(Dataset):
    def __init__(self, data, seq_len):
        super().__init__()
        self.data = data
        self.seq_len = seq_len

    def __getitem__(self, index):
        rand_start = torch.randint(0, self.data.size(0) - self.seq_len, (1,))
        full_seq = self.data[rand_start : rand_start + self.seq_len + 1].long()
        return full_seq.cuda()

    def __len__(self):
        return self.data.size(0) // self.seq_len


In [13]:
train_dataset = TextSamplerDataset(data_train, SEQ_LEN)
val_dataset = TextSamplerDataset(data_val, SEQ_LEN)
train_loader = cycle(DataLoader(train_dataset, batch_size=BATCH_SIZE))
val_loader = cycle(DataLoader(val_dataset, batch_size=BATCH_SIZE))

In [14]:
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [15]:
# training

for i in tqdm.tqdm(range(NUM_BATCHES), mininterval=10.0, desc="training"):
    model.train()

    for __ in range(GRADIENT_ACCUMULATE_EVERY):
        loss = model(next(train_loader))
        loss.backward()

    # print(f"training loss: {loss.item()}")
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optim.step()
    optim.zero_grad()

    if i % VALIDATE_EVERY == 0:
        print(f"training loss: {loss.item()}")
        model.eval()
        with torch.no_grad():
            loss = model(next(val_loader))
            print(f"validation loss: {loss.item()}")

    if i % GENERATE_EVERY == 0:
        model.eval()
        inp = random.choice(val_dataset)[:-1]
        prime = decode_tokens(inp)
        print((prime, "*" * 100))

        sample = model.generate(inp[None, ...], GENERATE_LENGTH)
        #print(sample[0])
        output_str = decode_tokens(sample[0])
        print(output_str)

training:   0%|                                                                              | 0/10000 [00:00<?, ?it/s]

training loss: 5.448845386505127
validation loss: 5.109672546386719
(['B7', 'EM7', 'E', 'B7'], '****************************************************************************************************')
['Adim7', 'C#dim']


training:   1%|▋                                                                    | 92/10000 [00:20<35:40,  4.63it/s]

training loss: 3.5636377334594727
validation loss: 3.5302047729492188


training:   2%|█▎                                                                  | 192/10000 [00:40<33:49,  4.83it/s]

...

training:  98%|█████████████████████████████████████████████████████████████████▌ | 9781/10000 [33:34<00:45,  4.85it/s]

training loss: 1.6337765455245972
validation loss: 3.973446846008301


training:  99%|██████████████████████████████████████████████████████████████████▏| 9879/10000 [33:54<00:24,  4.85it/s]

training loss: 1.9391199350357056
validation loss: 3.879714250564575


training: 100%|██████████████████████████████████████████████████████████████████| 10000/10000 [34:19<00:00,  4.86it/s]


In [16]:
import pickle
pickle.dump(model, open( "model.p", "wb"))

In [45]:
def chord_prog(prompt):
    inp = prompt.split(" ")
    inp = np.array([lookup[x] for x in inp], dtype=np.uint8)
    inp = torch.from_numpy(inp)
    inp = TextSamplerDataset(inp, SEQ_LEN)
    inp = inp[0]
    out = model.generate(inp[None, ...], 4)
    return decode_tokens(out[0])

In [ ]:
class TextSamplerDataset(Dataset):
    def __init__(self, data, seq_len):
        super().__init__()
        self.data = data
        self.seq_len = 3

    def __getitem__(self, index):
        rand_start = torch.randint(0, self.data.size(0) - self.seq_len, (1,))
        full_seq = self.data[rand_start : rand_start + self.seq_len + 1].long()
        return full_seq.cuda()

    def __len__(self):
        return self.data.size(0) // self.seq_len

In [51]:
for i in range(0,10):
    print(chord_prog("Am7 Dm7 Am7 Dm7"))

['Am7', 'Dm7', 'C7', 'C']
['Am7', 'Dm7', 'Am7', 'Dm7']
['C7', 'C', 'G', 'Em7']
['Am7', 'Dm7', 'Am7', 'Dm7']
['CM7', 'Dm7', 'Edim', 'Gm7']
['Am7', 'Dm7', 'Am7', 'Dm7']
['GM7', 'FM7', 'Em7', 'Gm7']
['C7', 'C', 'F#dim', 'F#dim']
['GM7', 'FM7', 'Em7', 'Gm7']
['Am7', 'Dm7', 'Am7', 'Dm7']
